In [1]:
from scikeras.wrappers import KerasClassifier

2024-10-29 02:20:50.466401: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-29 02:20:50.535579: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-29 02:20:50.535628: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-29 02:20:50.540259: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 02:20:50.549834: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-29 02:20:50.552632: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
import pandas  as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
import tensorflow as tf
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import  Dense

In [3]:
data = pd.read_csv('Churn_Modelling.csv')

In [4]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [5]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [8]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']])



In [9]:
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
## combine one hot encoded column with orginal data 
data = pd.concat([data.drop("Geography",axis=1),geo_encoded_df],axis=1) 
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


geo_encoded_df = pd.DataFrame(geo_encoded.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
## combine one hot encoded column with orginal data 
data = pd.concat([data.drop("Geography",axis=1),geo_encoded_df],axis=1)  

In [18]:
## divide the  data set into dependent and independent data set
X=data.drop('Exited',axis=1)
y = data['Exited']

## split the data in training and testing data
X_train ,X_test,y_train ,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
## scale these feature
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
with open('label_encoder_gender.pk1','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geo.pk1','wb') as file:
    pickle.dump(onehot_encoder_geo,file)
with open('scaler.pk1','wb') as file:
    pickle.dump(scaler,file)    

In [28]:
## define as function to create model and try different parameters (Keras Classifier)

def create_model(neurons =32 ,layers =1):
    model =Sequential()
    model.add(Dense(neurons,activation='relu',input_shape = (X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer = 'adam' ,loss ='binary_crossentropy',matrics = ['accuracy'])
    return model
    


In [29]:
## create a keras classifier
model = KerasClassifier(neurons =32 ,layers =1,build_fn=create_model,epochs=50,batch_size=10,verbose=0)


In [30]:
param_grid = {
    'neurons' : [16,32,64,128],
    'layers' : [1,2],
    'epochs' : [50,100]
}

In [31]:
## perform grid search
from sklearn.model_selection import GridSearchCV
 
grid = GridSearchCV(estimator= model ,param_grid =param_grid,n_jobs = -1 ,cv=3)
grid_result = grid.fit(X_train,y_train)

## print the best parameters
print("Best : %f using %s" % (grid_result.best_score_ , grid_result.best_params_))

2024-10-29 02:42:31.233516: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-29 02:42:31.233515: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-29 02:42:31.287016: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-29 02:42:31.287018: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-29 02:42:31.287070: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-29 02:42:31.287060: E external/local_xla/xla/stream_executor/cuda/cuda_f

ValueError: 
All the 48 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "/home/vscode/.local/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/vscode/.local/lib/python3.11/site-packages/scikeras/wrappers.py", line 1491, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/home/vscode/.local/lib/python3.11/site-packages/scikeras/wrappers.py", line 760, in fit
    self._fit(
  File "/home/vscode/.local/lib/python3.11/site-packages/scikeras/wrappers.py", line 915, in _fit
    X, y = self._initialize(X, y)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vscode/.local/li